# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

In [1]:
# try this project here!

In [2]:
import torch
import torch.optim as optim
from torch import nn, optim
import syft as sy
hook = sy.TorchHook(torch)

In [3]:
data = torch.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = torch.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [4]:
bob = sy.VirtualWorker(hook,id='bob')
alice = sy.VirtualWorker(hook,id='alice')
secure_worker = sy.VirtualWorker(hook,id='secure_worker')

In [5]:
bob.add_workers([alice , secure_worker])
alice.add_workers([bob,secure_worker])
secure_worker.add_workers([bob,alice])

<VirtualWorker id:secure_worker #objects:0>

In [6]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [7]:
data_alice = data[2:].send(alice)
target_alice = target[2:].send(alice)

In [8]:
model = nn.Linear(2,1)

In [9]:
for rounds in range(10):
    bob_model = model.copy().send(bob)
    alice_model = model.copy().send(alice)
    bob_opt = optim.SGD(bob_model.parameters(),lr = 0.01)
    alice_opt = optim.SGD(alice_model.parameters(),lr = 0.01)
    epoch = 10
    for i in range(epoch):
        bob_output = bob_model(data_bob)
        bob_loss = ((bob_output-target_bob)**2).sum()
        bob_opt.zero_grad()
        bob_loss.backward()
        bob_opt.step()


        alice_output = alice_model(data_alice)
        alice_loss = ((alice_output-target_alice)**2).sum()
        alice_opt.zero_grad()
        alice_loss.backward()
        alice_opt.step()
    bob_model.move(secure_worker)
    alice_model.move(secure_worker)
    with torch.no_grad():
        model.weight.set_(((bob_model.weight+alice_model.weight)/2).get())
        model.bias.set_(((bob_model.bias+alice_model.bias)/2).get())
    print( 'bob loss: ' + str(bob_loss.get()) +'alice loss: ' + str(alice_loss.get()))

bob loss: tensor(1.3830, requires_grad=True)alice loss: tensor(0.5034, requires_grad=True)
bob loss: tensor(0.5160, requires_grad=True)alice loss: tensor(0.0948, requires_grad=True)
bob loss: tensor(0.2161, requires_grad=True)alice loss: tensor(0.0363, requires_grad=True)
bob loss: tensor(0.1043, requires_grad=True)alice loss: tensor(0.0474, requires_grad=True)
bob loss: tensor(0.0586, requires_grad=True)alice loss: tensor(0.0619, requires_grad=True)
bob loss: tensor(0.0379, requires_grad=True)alice loss: tensor(0.0690, requires_grad=True)
bob loss: tensor(0.0274, requires_grad=True)alice loss: tensor(0.0699, requires_grad=True)
bob loss: tensor(0.0215, requires_grad=True)alice loss: tensor(0.0672, requires_grad=True)
bob loss: tensor(0.0178, requires_grad=True)alice loss: tensor(0.0627, requires_grad=True)
bob loss: tensor(0.0153, requires_grad=True)alice loss: tensor(0.0575, requires_grad=True)


# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [10]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [11]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [12]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!

In [13]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant. I.e.

In [14]:
x = 5

Q = 23740629843760239486723

bob_x_share = 23552870267 # <- a random number
alice_x_share = Q - bob_x_share + x
alice_x_share

23740629843736686616461

In [15]:
(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so.

In [16]:
x_share = (2,5,7)

Even though normally those shares would be distributed amongst several workers, you can store them in ordered tuples like this for now :)

In [17]:
# try this project here!

In [82]:
import random

In [83]:
Q = 38509385940350

In [146]:
def encrypt(value,shares):
    res = list()
    for i in range(shares-1):
        res.append(random.randint(0,Q))
        
    last = Q - (sum(res) % Q) + value
    res.append(last)
    res = tuple(res)
    return res

In [147]:
def decrypt(enc_tuple):
    return (sum(enc_tuple) % Q)

In [148]:
print(decrypt(encrypt(5,4)))

5


In [ ]:
def add(a,b):
    for i in range(len(a)):
        

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [25]:
BASE=10
PRECISION=4

In [26]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [27]:
encode(3.5)

35000

In [28]:
decode(35000)

3.5

In [29]:
x = encrypt(encode(5.5))
y = encrypt(encode(2.3))
z = add(x,y)
decode(decrypt(z))

7.8

# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [30]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [31]:
x = th.tensor([1,2,3,4,5])

### Secret Sharing Using PySyft

We can share using the simple .share() method!

In [32]:
x = x.share(bob, alice, secure_worker)

In [33]:
bob._objects

{35498656553: tensor([  10235770278698899, 1401398179551373756, 2277280072169145491,
          636965538565031298,  913795591610271305])}

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [34]:
y = x + x

In [35]:
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:23637986557 -> bob:30254176063]
	-> (Wrapper)>[PointerTensor | me:18229131498 -> alice:75856222543]
	-> (Wrapper)>[PointerTensor | me:34301722959 -> secure_worker:75419815101]
	*crypto provider: me*

In [36]:
y.get()

tensor([ 2,  4,  6,  8, 10])

### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using .fix_precision()

In [37]:
x = th.tensor([0.1,0.2,0.3])

In [38]:
x

tensor([0.1000, 0.2000, 0.3000])

In [39]:
x = x.fix_prec()

In [40]:
x.child.child

tensor([100, 200, 300])

In [41]:
y = x + x

In [42]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000])

### Shared Fixed Precision

And of course, we can combine the two!

In [43]:
x = th.tensor([0.1, 0.2, 0.3])

In [44]:
x = x.fix_prec().share(bob, alice, secure_worker)

In [45]:
y = x + x

In [46]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

Make sure to make the point that people can see the model averages in the clear.

# Final Project: Federated Learning with Encrypted Gradient Aggregation

In [155]:
data = torch.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = torch.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [168]:
bob.clear_objects()
alice.clear_objects()
secure_worker.clear_objects()

<VirtualWorker id:secure_worker #objects:0>

In [169]:
for rounds in range(10):
    bob_model = model.copy().send(bob)
    alice_model = model.copy().send(alice)
    bob_opt = optim.SGD(bob_model.parameters(),lr = 0.01)
    alice_opt = optim.SGD(alice_model.parameters(),lr = 0.01)
    epoch = 10
    for i in range(epoch):
        bob_output = bob_model(data_bob)
        bob_loss = ((bob_output-target_bob)**2).sum()
        bob_opt.zero_grad()
        bob_loss.backward()
        bob_opt.step()


        alice_output = alice_model(data_alice)
        alice_loss = ((alice_output-target_alice)**2).sum()
        alice_opt.zero_grad()
        alice_loss.backward()
        alice_opt.step()
    bob_model.move(secure_worker)
    alice_model.move(secure_worker)
    with torch.no_grad():
        model.weight.set_(((bob_model.weight+alice_model.weight)/2).get())
        model.bias.set_(((bob_model.bias+alice_model.bias)/2).get())
    print( 'bob loss: ' + str(bob_loss.get()) +'alice loss: ' + str(alice_loss.get()))

KeyError: 'Object "26387565303" not found on worker!!!You just tried to interact with an object ID:26387565303 on <VirtualWorker id:bob #objects:2> which does not exist!!! Use .send() and .get() on all your tensors to make sure they\'reon the same machines. If you think this tensor does exist, check the ._objects dictionaryon the worker and see for yourself!!! The most common reason this error happens is because someone calls.get() on the object\'s pointer without realizing it (which deletes the remote object and sends it to the pointer). Check your code to make sure you haven\'t already called .get() on this pointer!!!'